In [ ]:
# !git clone https://github.com/muxspace/facial_expressions.git

In [1]:
import csv
import os
from shutil import copyfile
import tensorflow as tf

from keras.preprocessing.image import ImageDataGenerator
# from keras.preprocessing.image import load_img
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout

In [2]:
data = {}
with open(r'C:\Users\che26\OneDrive\Programming\Python\Codes\5. DEEP LEARNING\Projects\3. Project Facial Emotions Recognition\data\legend.csv') as f:
    reader = csv.reader(f)
    next(reader)
    for row in reader:
        key = row[2].lower()
        if key in data:
            data[key].append(row[1])
        else:
            data[key] = [row[1]]

In [3]:
emotion_list = list(data.keys())

os.mkdir('./master_data')
os.mkdir('./master_data/training')
os.mkdir('./master_data/testing')

for emotion in emotion_list:
    os.mkdir(os.path.join('./master_data/training', emotion))
    os.mkdir(os.path.join('./master_data/testing', emotion))

split_size = 0.8
for emotion, images in data.items():
    train_size = int(split_size * len(images))
    train_images = images[:train_size]
    test_images = images[train_size:]
    for image in train_images:
        source = (os.path.join('./images', image))
        dest = os.path.join('./master_data/training',emotion, image)
        copyfile(source, dest)
    for image in test_images:
        source = os.path.join('./images', image)
        dest = os.path.join('./master_data/testing',emotion, image)
        copyfile(source, dest)

FileExistsError: [WinError 183] Cannot create a file when that file already exists: './master_data'

In [4]:
emotion_list

['anger',
 'surprise',
 'disgust',
 'fear',
 'neutral',
 'happiness',
 'sadness',
 'contempt']

In [5]:
model = tf.keras.models.Sequential([
    Conv2D(6, (3,3), activation = 'relu', input_shape = (28, 28, 1)),
    MaxPool2D(2, 2),
    Conv2D(16, (3,3), activation = 'relu'),
    MaxPool2D(2, 2),
    Flatten(),
    Dropout(0.5),
    Dense(200, activation = 'relu'),
    Dense(64, activation = 'relu'),
    Dense(8, activation = 'softmax')
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 6)         60        
                                                                 
 max_pooling2d (MaxPooling2  (None, 13, 13, 6)         0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 16)        880       
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 5, 5, 16)          0         
 g2D)                                                            
                                                                 
 flatten (Flatten)           (None, 400)               0         
                                                                 
 dropout (Dropout)           (None, 400)               0

In [6]:
model.compile( optimizer= Adam(learning_rate = 0.0001), loss = "categorical_crossentropy", metrics='accuracy')


In [7]:
train_dir = r'C:\Users\che26\OneDrive\Programming\Python\Codes\5. DEEP LEARNING\Projects\3. Project Facial Emotions Recognition\master_data\training'
test_dir = r'C:\Users\che26\OneDrive\Programming\Python\Codes\5. DEEP LEARNING\Projects\3. Project Facial Emotions Recognition\master_data\testing'

train_datagen = ImageDataGenerator(rescale = 1.0/255)
train_generator = train_datagen.flow_from_directory(
                    train_dir,
                    target_size = (28, 28),
                    class_mode='categorical',
                    batch_size=128,
                    color_mode='grayscale'
)

test_datagen = ImageDataGenerator(rescale = 1.0/255)
test_generator = test_datagen.flow_from_directory(
                    test_dir,
                    target_size = (28, 28),
                    class_mode='categorical',
                    batch_size=128,
                    color_mode='grayscale'
)

Found 10941 images belonging to 8 classes.
Found 2742 images belonging to 8 classes.


In [8]:
es = EarlyStopping(
                    monitor='loss',
                    patience=2,
                    min_delta=0.01
                   )

In [ ]:
model.fit(train_generator, epochs = 30, verbose = 1, callbacks = [es])

Epoch 1/30
15/86 [====>.........................] - ETA: 7s - loss: 2.0455 - accuracy: 0.2281

In [22]:
predictions = model.predict(test_generator)

28/28 [==============================] - 2s 72ms/step


In [23]:
import numpy as np
# prediction_value = []
# for i in predictions:
#     prediction_value.append(emotion_list[np.argmax(i)])
# print(prediction_value)
# print(len(prediction_value))
print(predictions[0])
print(predictions[1])
print(predictions[2])
print(predictions[3])

[0.06951588 0.00455687 0.04023398 0.01112701 0.4965248  0.31007645
 0.04797082 0.01999416]
[0.0110061  0.00056708 0.45395863 0.00568382 0.03957714 0.19967413
 0.27342173 0.01611141]
[5.2200500e-02 6.5405254e-04 2.1804050e-03 1.0106991e-03 2.5604313e-02
 8.2902694e-01 1.9169778e-02 7.0153281e-02]
[0.00973679 0.00083963 0.00365636 0.00217653 0.19829406 0.68410546
 0.02098562 0.08020546]
